# 6장 19대 대선 자료 분석하기

### 작성 : [PinkWink](http://pinkwink.kr) 

### 주의
* 이 책을 집필완료한 후 선거관리 위원회 홈페이지가 재보궐선거로 구성화면이 변경되었습니다.
* 그러므로 일일이 진입하던 코드를 생략하고, 바로 19대 대통령 선거 결과를 확인할 수 있는 페이지[바로가기](http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09)로 진입합니다.
* 그리고 6-1, 6-2절의 코드는 바뀝니다. 그러나 6-3절부터 학습하시는 경우라면 Github에서 배포하는 데이터를 사용하시면 됩니다.
* 아래 6-1, 6-2절의 코드가 변경되는 것을 양해해 주시기 바랍니다.

## 6-1. Selenium과 Beautiful Soup을 이용한 데이터 획득 준비 작업

In [1]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [2]:
from selenium import webdriver
import time

* 책에서 소개된 주소의 페이지는 이제 책 내용과 맞지 않습니다. P236의 In [3]은 아래와 같이 변경합니다.

In [11]:
driver = webdriver.Chrome('../driver/chromedriver')


OSError: [WinError 193] %1은(는) 올바른 Win32 응용 프로그램이 아닙니다

In [12]:
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

NameError: name 'driver' is not defined

* **대통령선거**라는 글자부분을 클릭합니다. 해당 글자에서 앞 장에서 설명한 크롬 개발자 도구로 코드를 확인하면 **id**가 **electionType1**으로 나타납니다.

In [6]:
driver.find_element_by_id("electionType1").click()

NameError: name 'driver' is not defined

* 그리고 나타나는 선택항목에서 **제19대**를 선택하도록 합니다. 역시 크롬 개발자 도구에서 해당 위치의 id는 **electionName**이라는 것을 찾을 수 있습니다.

In [7]:
driver.find_element_by_id("electionName").send_keys("제19대")

NameError: name 'driver' is not defined

* 그리고 나타나는 선택에서 **대통령선거**를 선택하도록 합니다.

In [ ]:
driver.find_element_by_id("electionCode").send_keys("대통령선거")

* 이제 나타나는 시도 항목에서 선택 부분의 XPath를 찾고, 해당 리스트를 가져옵니다

In [ ]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

## 6-2. 19대 대선 개표 결과 데이터 획득하기

<img src="../img/002.jpg" width="600" height="400"></img>

* 위 그림이 지역을 선택하면 나타나는 화면중 하나인데 득표수에 득표율이 괄호()로 함께 나타납니다.
* 이를 제거하고 **(**를 기준으로 왼쪽 숫자만 얻어서, 콤마(,)를 제거하고, float형으로 변경하는 함수를 **get_num**으로 준비해 둡니다

In [ ]:
import re

def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

* 책에서는 특별히 다루고 있지 않지만, 아래 모듈을 import해서 wait.until 함수를 사용할 수 있습니다.

<img src="../img/003.jpg" width="600" height="200"></img>

* 이 함수는 **검색**버튼이 클릭가능할 때 까지 기다리는 기능을 합니다.
* 그리고 **move_sido** 함수는 광역시도 이름을 리스트에 전송하고 검색 버튼을 누르는 역할을 합니다.

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click() 

* **append_data** 함수는 빈 내용으로 미리 준비한 DataFrame에 append 명령으로 읽은 데이터를 하나씩 추가하는 기능입니다.

<img src="../img/002.jpg" width="600" height="400"></img>

* 이 함수가 실행되면 전체 투표수, 문재인후보, 홍준표후보, 안철수후보의 득표수가 추가됩니다

In [ ]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(get_num(each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

* 미리 변수를 하나 만들어 둡니다.

In [ ]:
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

* 교재를 집필할 때는 table로 읽는 것이 부담된다고 판단했는데, 지금은 형태가 바뀌어서 html 태그 중 table로 되어 있는 부분을 Beautiful Soup으로 읽어서 padnas의 read_html로 읽는 것이 더 유리합니다.

In [ ]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

In [ ]:
election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

In [ ]:
election_result.to_csv('../data/05. election_result.csv', encoding='utf-8', sep=',')

In [ ]:
driver.close()

## 6-3. 각 후보의 득표율과 지역 ID 정리하기

In [ ]:
election_result = pd.read_csv('../data/05. election_result.csv', encoding='utf-8', 
                              index_col=0)
election_result.head()

In [ ]:
sido_candi =  election_result['광역시도']
sido_candi = [name[:2] if name[:2] 
                                          in ['서울','부산','대구','광주','인천','대전','울산']
                                    else '' for name in sido_candi]

In [ ]:
def cut_char_sigu(name):
    return name if len(name)==2 else name[:-1]

In [ ]:
import re

sigun_candi = ['']*len(election_result)

for n in election_result.index:
    each = election_result['시군'][n]
    if each[:2] in ['수원', '성남','안양','안산','고양',
                            '용인','청주','천안','전주','포항','창원']:
        sigun_candi[n] = re.split('시', each)[0]+' '+ \
                                                        cut_char_sigu(re.split('시', each)[1])
    else:
        sigun_candi[n] = cut_char_sigu(each)
        
sigun_candi

In [ ]:
ID_candi = [sido_candi[n]+' '+sigun_candi[n] for n in range(0,len(sigun_candi))]

ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi]

ID_candi

In [ ]:
election_result['ID'] = ID_candi
election_result.head(10)

In [ ]:
election_result[['rate_moon','rate_hong','rate_ahn']] = \
            election_result[['moon','hong','ahn']].div(election_result['pop'],  axis=0)
election_result[['rate_moon','rate_hong','rate_ahn']] *= 100
election_result.head()

In [ ]:
election_result.sort_values(['rate_moon'], ascending=[False]).head(10)

In [ ]:
election_result.sort_values(['rate_hong'], ascending=[False]).head(10)

In [ ]:
election_result.sort_values(['rate_ahn'], ascending=[False]).head(10)

In [ ]:
draw_korea = pd.read_csv('../data/05. draw_korea.csv', encoding='utf-8', 
                                                                                                 index_col=0)
draw_korea.head()

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
election_result[election_result['ID'] == '고성']

In [ ]:
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군'] == '고성군']

In [ ]:
election_result[election_result['광역시도'] == '경상남도']

In [ ]:
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'

election_result[election_result['광역시도'] == '경상남도']

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
election_result[election_result['시군'] == '부천시']

In [ ]:
election_result.tail()

In [ ]:
ahn_tmp = election_result.loc[85, 'ahn']/3
hong_tmp = election_result.loc[85, 'hong']/3
moon_tmp = election_result.loc[85, 'moon']/3
pop_tmp = election_result.loc[85, 'pop']/3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 소사', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 오정', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[252] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 원미', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]

In [ ]:
election_result[election_result['시군'] == '부천시']

In [ ]:
election_result.drop([85], inplace=True)
election_result[election_result['시군'] == '부천시']

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

In [ ]:
final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - \
                                                                final_elect_data['rate_hong']
final_elect_data.head()

In [ ]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[False]).head(10)

In [ ]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[True]).head(10)

## 6-4. 19대 대선 결과 득표율 시각화하기

In [ ]:
BORDER_LINES = [
    [(5, 1), (5,2), (7,2), (7,3), (11,3), (11,0)], # 인천
    [(5,4), (5,5), (2,5), (2,7), (4,7), (4,9), (7,9), 
     (7,7), (9,7), (9,5), (10,5), (10,4), (5,4)], # 서울
    [(1,7), (1,8), (3,8), (3,10), (10,10), (10,7), 
     (12,7), (12,6), (11,6), (11,5), (12, 5), (12,4), 
     (11,4), (11,3)], # 경기도
    [(8,10), (8,11), (6,11), (6,12)], # 강원도
    [(12,5), (13,5), (13,4), (14,4), (14,5), (15,5), 
     (15,4), (16,4), (16,2)], # 충청북도
    [(16,4), (17,4), (17,5), (16,5), (16,6), (19,6), 
     (19,5), (20,5), (20,4), (21,4), (21,3), (19,3), (19,1)], # 전라북도
    [(13,5), (13,6), (16,6)], # 대전시
    [(13,5), (14,5)], #세종시
    [(21,2), (21,3), (22,3), (22,4), (24,4), (24,2), (21,2)], #광주
    [(20,5), (21,5), (21,6), (23,6)], #전라남도
    [(10,8), (12,8), (12,9), (14,9), (14,8), (16,8), (16,6)], #충청북도
    [(14,9), (14,11), (14,12), (13,12), (13,13)], #경상북도
    [(15,8), (17,8), (17,10), (16,10), (16,11), (14,11)], #대구
    [(17,9), (18,9), (18,8), (19,8), (19,9), (20,9), (20,10), (21,10)], #부산
    [(16,11), (16,13)], #울산
    [(27,5), (27,6), (25,6)],
]

In [ ]:
def drawKorea(targetData, blockedMap, cmapname):
    gamma = 0.75

    whitelabelmin = 20.

    datalabel = targetData

    tmp_max = max([ np.abs(min(blockedMap[targetData])), 
                                  np.abs(max(blockedMap[targetData]))])
    vmin, vmax = -tmp_max, tmp_max

    mapdata = blockedMap.pivot_table(index='y', columns='x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(9, 11))
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=cmapname, 
               edgecolor='#aaaaaa', linewidth=0.5)

    # 지역 이름 표시
    for idx, row in blockedMap.iterrows():
        # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다. 
        #(중구, 서구)
        if len(row['ID'].split())==2:
            dispname = '{}\n{}'.format(row['ID'].split()[0], row['ID'].split()[1])
        elif row['ID'][:2]=='고성':
            dispname = '고성'
        else:
            dispname = row['ID']

        # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 10.0, 1.1
        else:
            fontsize, linespacing = 11, 1.

        annocolor = 'white' if np.abs(row[targetData]) > whitelabelmin else 'black'
        plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                     fontsize=fontsize, ha='center', va='center', color=annocolor,
                     linespacing=linespacing)

    # 시도 경계 그린다.
    for path in BORDER_LINES:
        ys, xs = zip(*path)
        plt.plot(xs, ys, c='black', lw=2)

    plt.gca().invert_yaxis()

    plt.axis('off')

    cb = plt.colorbar(shrink=.1, aspect=10)
    cb.set_label(datalabel)

    plt.tight_layout()
    plt.show()

In [ ]:
drawKorea('moon_vs_hong', final_elect_data, 'RdBu')

In [ ]:
drawKorea('moon_vs_ahn', final_elect_data, 'RdBu')

In [ ]:
drawKorea('ahn_vs_hong', final_elect_data, 'RdBu')

In [ ]:
import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
pop_folium = final_elect_data.set_index('ID')

del pop_folium['광역시도']
del pop_folium['시군']

pop_folium.head()

In [ ]:
geo_path = '../data/05. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

map = folium.Map(location=[36.2002, 127.054], zoom_start=6)
map.choropleth(geo_data = geo_str,
               data = pop_folium['moon_vs_hong'],
               columns = [pop_folium.index, pop_folium['moon_vs_hong']],
               fill_color = 'PuBu', #PuRd, YlGnBu
               key_on = 'feature.id')

map